In [1]:
# Importamos librerias

import torch
import torch.nn as nn
import sys
import os
import numpy as np 

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
from sklearn.model_selection import train_test_split

import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm 

print("Liberias importadas correctamente")

Liberias importadas correctamente


In [2]:
# Modelo YOLO Version 3 para detección de objetos (celdas de sangre)
# Basado en el repositorio de Manuel Garcia UEM Junio 2025

#1. Reproducibilidad total

import random
import numpy as np
import torch

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
print(f"Semilla global fijada en {SEED}")

Semilla global fijada en 1234


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True) 



In [5]:
# Funcion de calculo de metricas de evaluación IoU, Precisión, Recall y mAP (dummy function)

def calculate_iou(box1, box2):
    # box: (cx, cy, w, h) formato normalizado [0,1]
    x1_min = box1[0] - box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_max = box1[1] + box1[3] / 2

    x2_min = box2[0] - box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_max = box2[1] + box2[3] / 2

    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)
    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    union = area1 + area2 - inter_area + 1e-12
    return inter_area / union

def simple_epoch_metrics(model, dataloader, device, iou_threshold=0.5):
    model.eval()
    total_iou = []
    tp, fp, fn = 0, 0, 0
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            # Aquí deberías obtener las predicciones decodificadas de tu modelo real
            # Por ahora, usamos las GT como si fueran predicciones (dummy)
            batch_size = len(targets)
            for i in range(batch_size):
                gt_boxes = targets[i].cpu().numpy()
                pred_boxes = gt_boxes  # REEMPLAZA esto por tus predicciones reales decodificadas
                matched_gt = set()
                for pb in pred_boxes:
                    best_iou = 0
                    best_idx = -1
                    for idx, gb in enumerate(gt_boxes):
                        iou = calculate_iou(pb[1:], gb[1:])  # omite la clase
                        if iou > best_iou:
                            best_iou = iou
                            best_idx = idx
                    total_iou.append(best_iou)
                    if best_iou >= iou_threshold and best_idx not in matched_gt:
                        tp += 1
                        matched_gt.add(best_idx)
                    else:
                        fp += 1
                fn += len(gt_boxes) - len(matched_gt)
    mean_iou = np.mean(total_iou) if total_iou else 0.0
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    mAP = mean_iou  # (Para propósitos de ejemplo, igual que mean IoU)
    return mean_iou, precision, recall, mAP

In [ ]:
# ==== EVALUACIÓN FINAL EN EL SET DE TEST ====
print("\n--- Comenzando evaluación final en el set de test ---")
# Vuelve a cargar el mejor modelo (por si el modelo en memoria no es el mejor tras early stopping)
model.load_state_dict(torch.load("best_yolov3_bloodcell.pth", map_location=device))
print("Modelo cargado desde best_yolov3_bloodcell.pth")
model.eval()

# Calcula métricas en el test set
mean_iou_test, precision_test, recall_test, mAP_test = simple_epoch_metrics(model, test_loader, device)
print(f"[Test] IoU: {mean_iou_test:.4f} | Precision: {precision_test:.4f} | Recall: {recall_test:.4f} | mAP: {mAP_test:.4f}")

# Loguea en TensorBoard (opcional)
writer = SummaryWriter(log_dir='runs/yolov3_bloodcell')
writer.add_scalar("Test/IoU", mean_iou_test)
writer.add_scalar("Test/Precision", precision_test)
writer.add_scalar("Test/Recall", recall_test)
writer.add_scalar("Test/mAP", mAP_test)
writer.close()

In [6]:
# Función de decodificación de predicciones YOLOv3

def yolo_decode_outputs(outputs, anchors, num_classes, img_size, conf_thres=0.3, iou_thres=0.5, device='cpu', debug=False):
    """
    Decodifica las predicciones de YOLOv3 (PyTorch) y aplica NMS.
    Adapta automáticamente la forma del tensor (aplanado o no).
    """
    from torchvision.ops import nms
    batch_size = outputs[0].shape[0]
    final_preds = [[] for _ in range(batch_size)]
    for scale_idx, output in enumerate(outputs):
        if debug:
            print(f"DEBUG: outputs[{scale_idx}].shape = {output.shape}")
        anchor_set = torch.tensor(anchors[scale_idx], dtype=torch.float32, device=device)
        nA = len(anchors[scale_idx])
        N = output.shape[1] if output.dim() == 3 else None
        # Deducimos grid_size
        if output.dim() == 3:
            # Caso [batch, N, 5+C] (aplanado)
            grid_size = int((output.shape[1] // nA) ** 0.5)
            if output.shape[1] != nA * grid_size * grid_size:
                raise ValueError(f"Shape inesperado: {output.shape}")
            output = output.view(batch_size, nA, grid_size, grid_size, 5 + num_classes)
            if debug:
                print(f" -> reshape a [{batch_size},{nA},{grid_size},{grid_size},{5+num_classes}]")
        elif output.dim() == 5:
            # Caso ya en [batch, nA, grid, grid, 5+C]
            grid_size = output.shape[2]
            # No hace falta reshape
        else:
            raise ValueError(f"Shape no soportado: {output.shape}")
        output = output.cpu()  # para facilitar
        for b in range(batch_size):
            pred = output[b]  # [nA, grid, grid, 5+C]
            pred = pred.permute(0, 2, 3, 1).contiguous().view(-1, 5 + num_classes)  # [nA*grid*grid, 5+C]
            # Sigmoid para cx, cy, conf, clases
            pred_boxes = pred[:, :4]
            pred_obj = pred[:, 4].sigmoid()
            pred_cls = pred[:, 5:].sigmoid()
            pred[:, 4:] = torch.cat([pred_obj.unsqueeze(1), pred_cls], 1)
            # Filtra por confianza mínima
            mask = pred_obj > conf_thres
            pred = pred[mask]
            if pred.shape[0] == 0:
                continue
            # Encuentra clase más probable y conf
            class_confs, class_ids = pred[:, 5:].max(1)
            scores = pred[:, 4] * class_confs
            cx, cy, w, h = pred[:, 0], pred[:, 1], pred[:, 2], pred[:, 3]
            # Las coord. suelen estar normalizadas a [0,1]
            x1 = cx - w/2
            y1 = cy - h/2
            x2 = cx + w/2
            y2 = cy + h/2
            boxes_xyxy = torch.stack([x1, y1, x2, y2], 1)
            keep = nms(boxes_xyxy, scores, iou_thres)
            for idx in keep:
                final_preds[b].append([
                    int(class_ids[idx].item()),
                    cx[idx].item(),
                    cy[idx].item(),
                    w[idx].item(),
                    h[idx].item(),
                    scores[idx].item()
                ])
    final_preds = [np.array(p) if len(p) else np.zeros((0,6)) for p in final_preds]
    return final_preds

print("Función de decodificación de predicciones YOLOv3 definida correctamente.")



Función de decodificación de predicciones YOLOv3 definida correctamente.


In [7]:
# Predicciones y visualización de resultados

import cv2
import matplotlib.pyplot as plt

def show_yolo_predictions(
    model, dataloader, anchors, num_classes, class_names,
    device, img_size=416, conf_threshold=0.3, iou_threshold=0.5,
    max_images=10, show_gt=True
):
    """
    Visualiza las predicciones del modelo YOLOv3 sobre imágenes reales.
    - model: tu modelo
    - dataloader: DataLoader (test o val)
    - anchors, num_classes: como en tu pipeline
    - class_names: lista de strings de nombres de clase
    - device: cpu o cuda
    - img_size: tamaño de entrada del modelo
    - conf_threshold: umbral de confianza para mostrar predicciones
    - iou_threshold: NMS
    - max_images: máximo de imágenes a visualizar
    - show_gt: si True, dibuja también las GT
    """
    model.eval()
    cmap = plt.get_cmap('tab10')
    with torch.no_grad():
        shown = 0
        for images, targets in dataloader:
            images = images.to(device)
            outputs = model(images)
            pred_boxes = yolo_decode_outputs(
                outputs, anchors, num_classes, img_size=img_size,
                conf_thres=conf_threshold, iou_thres=iou_threshold, device=device
            )
            for i in range(images.shape[0]):
                img_np = images[i].cpu().permute(1,2,0).numpy()
                # Desnormaliza si usaste normalización estándar
                img_np = img_np * np.array([0.229,0.224,0.225]) + np.array([0.485,0.456,0.406])
                img_np = (img_np * 255).clip(0,255).astype(np.uint8)
                img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

                # Dibuja predicciones
                for pb in pred_boxes[i]:
                    class_id = int(pb[0])
                    cx, cy, w, h, conf = pb[1], pb[2], pb[3], pb[4], pb[5]
                    x1 = int((cx - w/2) * img_bgr.shape[1])
                    y1 = int((cy - h/2) * img_bgr.shape[0])
                    x2 = int((cx + w/2) * img_bgr.shape[1])
                    y2 = int((cy + h/2) * img_bgr.shape[0])
                    color = tuple([int(255*x) for x in cmap(class_id)[:3]])
                    cv2.rectangle(img_bgr, (x1,y1), (x2,y2), color, 2)
                    label = f"{class_names[class_id]} {conf:.2f}"
                    cv2.putText(img_bgr, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Dibuja GT (opcional)
                if show_gt and len(targets[i]) > 0:
                    for gt in targets[i]:
                        class_id = int(gt[0].item())
                        cx, cy, w, h = gt[1:].tolist()
                        x1 = int((cx - w/2) * img_bgr.shape[1])
                        y1 = int((cy - h/2) * img_bgr.shape[0])
                        x2 = int((cx + w/2) * img_bgr.shape[1])
                        y2 = int((cy + h/2) * img_bgr.shape[0])
                        color = (0,255,0)  # Verde para GT (puedes cambiarlo)
                        cv2.rectangle(img_bgr, (x1,y1), (x2,y2), color, 1)
                        cv2.putText(img_bgr, class_names[class_id], (x1, y2+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

                # Muestra la imagen
                plt.figure(figsize=(8,8))
                plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
                plt.title('Predicciones (rectángulo grueso) / GT (verde fino)')
                plt.axis('off')
                plt.show()
                shown += 1
                if shown >= max_images:
                    return

# Ejemplo de uso:

class_names = ['RBC', 'WBC', 'Platelets']
show_yolo_predictions(
    model, test_loader, anchors=ANCHORS, num_classes=NUM_CLASSES,
    class_names=class_names, device=device, img_size=416,
    conf_threshold=0.3, iou_threshold=0.5, max_images=5, show_gt=True
)

NameError: name 'model' is not defined